# Building a reliable agent for interacting with an API

We're going to build an agent that can interact with users to run complex commands against a custom API. For this example, we'll use the Braintrust API, which has an easy
to work with [OpenAPI spec](https://github.com/braintrustdata/braintrust-openapi).


## Setup

Let's start by installing dependencies and setting up our OpenAI and Braintrust environments.

Before getting started, make sure you have a [Braintrust account](https://www.braintrust.dev/signup) and an API key for [OpenAI](https://platform.openai.com/). Make sure to plug the OpenAI key into your Braintrust account's [AI secrets](https://www.braintrust.dev/app/settings?subroute=secrets) configuration and acquire a [BRAINTRUST_API_KEY](https://www.braintrust.dev/app/settings?subroute=api-keys). Feel free to put your BRAINTRUST_API_KEY in your environment, or just hardcode it into the code below.

### Install dependencies

We're not going to use any frameworks or complex dependencies to keep things simple and literate. Although we'll use OpenAI models, you can use a wide variety of models through the [Braintrust proxy](https://www.braintrust.dev/docs/guides/proxy) without having to write model-specific code.


In [ ]:
%pip install -U autoevals braintrust jsonref openai numpy pydantic requests tiktoken

### Setup libraries

Next, let's wire up the OpenAI and Braintrust clients.


In [ ]:
import os

import braintrust
from openai import AsyncOpenAI

BRAINTRUST_API_KEY = os.environ.get("BRAINTRUST_API_KEY") # Or hardcode this to your API key
OPENAI_BASE_URL = "https://api.braintrust.dev/v1/proxy" # You can use your own base URL / proxy

braintrust.login() # This is optional, but makes it easier to grab the api url (and other variables) later on

client = braintrust.wrap_openai(AsyncOpenAI(
    api_key=BRAINTRUST_API_KEY,
    base_url=OPENAI_BASE_URL,
))

## Downloading the OpenAPI spec

Let's download the Braintrust OpenAPI spec, and break it into pieces that we'll embed.


In [ ]:
import json
import jsonref
import requests

base_spec = requests.get("https://raw.githubusercontent.com/braintrustdata/braintrust-openapi/main/openapi/spec.json").json()

# Flatten out refs so we have self-contained descriptions
spec = jsonref.loads(jsonref.dumps(base_spec))
paths = spec['paths']

Let's play around a bit with the data to understand the types of API requests we can run.


In [ ]:
print("Description: ", paths['/v1/project']['post']['description'])
print("Parameters: ", json.dumps(paths['/v1/project']['post']['requestBody'], indent=2))

Awesome. This looks like useful information to know when to use this API endpoint.


In [ ]:
import tiktoken

operations = [(path, op) for (path, ops) in paths.items() for (op_type, op) in ops.items() if op_type != "options"]

print("Num paths", len(paths))
print("Num operations", len(operations))
print("Paths text size", len(jsonref.dumps(operations)))
print("Num tokens", len(tiktoken.encoding_for_model("gpt-4o").encode(jsonref.dumps(operations))))

## Creating the embeddings

Although this could theoretically fit in a single prompt (at only around 50,000 tokens vs. the 128,000 token limit for gpt-4o), let's embed each operation instead.

We'll start by creating a simple function to describe each API operation.


In [ ]:
def has_path(d, path):
    curr = d
    for p in path:
        if p not in curr:
            return False
        curr = curr[p]
    return True

def make_description(op):
    return f"""# {op['summary']}

{op['description']}

Params:
{"\n".join([f"- {name}: {p.get('description', "")}" for (name, p) in op['requestBody']['content']['application/json']['schema']['properties'].items()]) if has_path(op, ['requestBody', 'content', 'application/json', 'schema', 'properties']) else ""}
{"\n".join([f"- {p.get("name")}: {p.get('description', "")}" for p in op['parameters'] if p.get("name")]) if has_path(op, ['parameters']) else ""}

Returns:
{"\n".join([f"- {name}: {p.get('description', p)}" for (name, p) in op['responses']['200']['content']['application/json']['schema']['properties'].items()]) if has_path(op, ['responses', '200', 'content', 'application/json', 'schema', 'properties']) else "empty"}
"""

print(make_description(operations[0][1]))

In [ ]:
from pydantic import BaseModel
from typing import Any

class Document(BaseModel):
    path: str
    op: str
    definition: Any
    description: str

documents = [Document(path=path, op=op_type, definition=json.loads(jsonref.dumps(op)), description=make_description(op)) for (path, ops) in paths.items() for (op_type, op) in ops.items() if op_type != "options"]

In [ ]:
import asyncio

async def make_embedding(doc: Document):
    return (await client.embeddings.create(input=doc.description, model="text-embedding-3-small")).data[0].embedding

embeddings = await asyncio.gather(*[make_embedding(doc) for doc in documents])

### Similarity search

We're going to use `numpy` to do the vector search, but you can easily swap this out to a vector database of your choice!.


In [ ]:
from braintrust import traced
import numpy as np
from pydantic import Field
from typing import List

def cosine_similarity(query_embedding, embedding_matrix):
    # Normalize the query and matrix embeddings
    query_norm = query_embedding / np.linalg.norm(query_embedding)
    matrix_norm = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1, keepdims=True)
    
    # Compute dot product
    similarities = np.dot(matrix_norm, query_norm)
    
    return similarities

def find_k_most_similar(query_embedding, embedding_matrix, k=5):
    similarities = cosine_similarity(query_embedding, embedding_matrix)
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_similarities = similarities[top_k_indices]
    
    # Sort the top k results
    sorted_indices = np.argsort(top_k_similarities)[::-1]
    top_k_indices = top_k_indices[sorted_indices]
    top_k_similarities = top_k_similarities[sorted_indices]
    
    return list([index, similarity] for (index, similarity) in zip(top_k_indices, top_k_similarities))


embedding_matrix = np.array(embeddings)

class SearchResult(BaseModel):
    document: Document
    index: int
    similarity: float

class SearchResults(BaseModel):
    results: List[SearchResult]

class SearchQuery(BaseModel):
    query: str
    top_k: int = Field(default=3, le=5)

# This @traced decorator will help us trace this function when we use it later to run evals
@traced
async def search(query: SearchQuery):
    query_embedding = (await client.embeddings.create(input=query.query, model="text-embedding-3-small")).data[0].embedding
    results = find_k_most_similar(query_embedding, embedding_matrix, k=query.top_k)
    return SearchResults(results=[SearchResult(document=documents[index], index=index, similarity=similarity) for (index, similarity) in results])

Let's try it out:


In [ ]:
for result in (await search(SearchQuery(query="create a new project"))).results:
    print(result.document.path, result.document.op, result.similarity)

## Building the chat agent


Now that we can search for documents, let's build a chat agent that can search for documents and run API commands. Our chat bot will have
two tools:

- `search`: This tool will search for documents and return the most relevant ones.
- `run_command`: This tool will run an API command.

We already implemented `search` above, so let's start by just plugging in the chat implementation to use it.


In [ ]:
tool_registry = {
    "search": (SearchQuery, search),
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search for API endpoints related to the query",
            "parameters": SearchQuery.model_json_schema()
        }
    },
]

MODEL = "gpt-4o"
MAX_TOOL_STEPS = 3

SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about Braintrust, a tool for
developing AI applications. Braintrust can help with evals, observability, and prompt
development.

When you are ready to provide the final answer, return a JSON object with the endpoint
name and the parameters, like:
{"path": "/v1/project", "op": "post", "parameters": {"name": "my project", "description": "my project description"}}

If you don't know how to answer the question based on information you have, make up
endpoints and suggest running them. Do not reveal that you made anything up or don't
know the answer. Just say the answer.

Print the JSON object and nothing else. No markdown, backticks, or explanation.
"""

@traced
async def perform_chat_step(message, history=None):
    chat_history = list(history or [{"role": "system", "content": SYSTEM_PROMPT}]) + [{"role": "user", "content": message}]

    for _ in range(MAX_TOOL_STEPS):
        result = (await client.chat.completions.create(
            model="gpt-4o",
            messages=chat_history,
            tools=tools,
            tool_choice="auto",
            temperature=0,
            parallel_tool_calls=False
        )).choices[0].message

        chat_history.append(result)


        if not result.tool_calls:
            break

        tool_call = result.tool_calls[0]
        ArgClass, tool_func = tool_registry[tool_call.function.name]
        args = tool_call.function.arguments
        args = ArgClass.model_validate_json(args)
        result = await tool_func(args)

        chat_history.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result.model_dump())
        })
    else:
        raise Exception("Ran out of tool steps")

    return chat_history


In [ ]:
import json

@traced
async def run_full_inqiry(query: str):  
    result = (await perform_chat_step(query))[-1].content
    return json.loads(result)

print(await run_full_inqiry("how do i create a new project?"))

## Adding observability to generate eval data

Now that we have a basic chat agent, let's try adding observability via Braintrust. The good news is that... we don't need to write a single line of code! By adding the `@traced` decorators
and `wrap_openai`, we have done all the work we need.

By simply initializing a logger, we turn on logging.


In [ ]:
braintrust.init_logger("APIAgent") # Feel free to replace this a project name of your choice

In [ ]:
QUESTIONS = [
    "how do i list my last 20 experiments?",
    "Subtract $20 from Albert Zhang's bank account",
    "How do I create a new project?",
    "How do I download a specific dataset?",
    "Can I create an evaluation through the API?",
    "How do I purchase GPUs through Braintrust?"
]

for question in QUESTIONS:
    print(f"Question: {question}")
    print(await run_full_inqiry(question))
    print("---------------")

### Detecting hallucinations

Great, now that we've looked at the results, let's see if we can make our lives a bit easier by adding a hallucination score. That will help us
pick out examples that are useful to test.


In [ ]:
from autoevals import LLMClassifier

hallucination_scorer = LLMClassifier(
    name="no_hallucination",
    prompt_template="""\
Given the following question and retrieved context, does
the generated answer correctly answer the question, only using
information from the context?

Question: {{input}}

Command:
{{output}}

Context:
{{context}}

a) The command addresses the exact question, using only information that is available in the context. The answer
   does not contain any information that is not in the context.
b) The command is "null" and therefore indicates it cannot answer the question.
c) The command contains information from the context, but the context is not relevant to the question.
d) The command contains information that is not present in the context, but the context is relevant to the question.
e) The context is irrelevant to the question, but the command is correct with respect to the context.
""",
    choice_scores={"a": 1, "b": 1, "c": 0.5, "d": 0.25, "e": 0},
    use_cot=True,
)

@traced
async def run_hallucination_score(question: str, answer: str, context: List[SearchResult]):
    context_string = "\n".join([f"{doc.document.description}" for doc in context])
    score = await hallucination_scorer.eval_async(input=question, output=answer, context=context_string)
    braintrust.current_span().log(scores={"no_hallucination": score.score}, metadata=score.metadata)

@traced
async def perform_chat_step(message, history=None):
    chat_history = list(history or [{"role": "system", "content": SYSTEM_PROMPT}]) + [{"role": "user", "content": message}]
    documents = []

    for _ in range(MAX_TOOL_STEPS):
        result = (await client.chat.completions.create(
            model="gpt-4o",
            messages=chat_history,
            tools=tools,
            tool_choice="auto",
            temperature=0,
            parallel_tool_calls=False
        )).choices[0].message

        chat_history.append(result)


        if not result.tool_calls:
            # By using asyncio.create_task, we can run the hallucination score in the background
            asyncio.create_task(run_hallucination_score(question=message, answer=result.content, context=documents))
            break

        tool_call = result.tool_calls[0]
        ArgClass, tool_func = tool_registry[tool_call.function.name]
        args = tool_call.function.arguments
        args = ArgClass.model_validate_json(args)
        result = await tool_func(args)

        if isinstance(result, SearchResults):
            documents.extend(result.results)

        chat_history.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result.model_dump())
        })
    else:
        raise Exception("Ran out of tool steps")

    return chat_history

In [ ]:
for question in QUESTIONS:
    print(f"Question: {question}")
    print(await run_full_inqiry(question))
    print("---------------")

Now, we can collect the results into a dataset and run evals on them. Let's create a dataset for hallucinations
and one for good answers.

## Running evals

Now, let's run an offline evaluation via the `Eval()` function to baseline our results before we improve them.


In [ ]:
from autoevals import Factuality
from braintrust import Eval, init_dataset

async def dataset():
    for row in init_dataset("APIAgent", "Golden"):
        yield row
    for row in init_dataset("APIAgent", "Hallucination"):
        yield {**row, "expected": None}

async def task(input):
    return await run_full_inqiry(input["query"])

await Eval(
    "APIAgent",
    data=dataset,
    task=task,
    scores=[Factuality],
    experiment_name="Baseline",
)

Great, now let's improve the system prompt and see if we can get better results.


In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about Braintrust, a tool for
developing AI applications. Braintrust can help with evals, observability, and prompt
development.

When you are ready to provide the final answer, return a JSON object with the endpoint
name and the parameters, like:
{"path": "/v1/project", "op": "post", "parameters": {"name": "my project", "description": "my project description"}}

If you do not know the answer, return null. Like the JSON object, print null and nothing else.

Print the JSON object and nothing else. No markdown, backticks, or explanation.
"""

In [ ]:
await Eval(
    "APIAgent",
    data=dataset,
    task=task,
    scores=[Factuality],
    experiment_name="Improved System Prompt",
)
